In [1]:
import $file.^.sparkinit, sparkinit._
import $file.^.pathinit, pathinit._
import $file.^.cpinit, cpinit._
import ss.implicits._
import com.relatedsciences.opentargets.etl.pipeline.Functions
import org.apache.spark.sql.functions._
import java.nio.file.Paths
import org.apache.spark.sql.DataFrame
import org.apache.spark.storage.StorageLevel

Loading spark-stubs
Creating SparkSession


Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
19/12/17 20:13:18 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


import $file.$          , sparkinit._

import $file.$         , pathinit._

import $file.$       , cpinit._

import ss.implicits._

import com.relatedsciences.opentargets.etl.pipeline.Functions

import org.apache.spark.sql.functions._

import java.nio.file.Paths

import org.apache.spark.sql.DataFrame

import org.apache.spark.storage.StorageLevel

In [2]:
def prefix(prefix: String)(df: DataFrame): DataFrame = {
  val cols = df.columns.map(c => df(c).as(s"$prefix$c"))
  df.select(cols: _*)
}

defined function prefix

### Load Results (from v1 and v2 pipelines)

In [3]:
val dfv2 = ss.read.parquet(RESULTS_DIR.resolve("evidence_raw.parquet").toString)
    // TODO: remove this once the pipeline has been rerun
    .transform(Functions.addEvidenceRecordId("id"))

parquet at cmd2.sc:1

1 / 1

19/12/17 20:13:26 WARN Utils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.debug.maxToStringFields' in SparkEnv.conf.


dfv2: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [access_level: string, disease: struct<biosample: struct<id: string, name: string>, id: string ... 2 more fields> ... 11 more fields]

In [4]:
val dfv1 = ss.read.json(EXTRACT_DIR.resolve("evidence.json").toString)
    .transform(Functions.addEvidenceRecordId("id"))

json at cmd3.sc:1

115 / 115

dfv1: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [access_level: string, disease: struct<biosample: struct<id: string, name: string>, efo_info: struct<efo_id: string, label: string ... 2 more fields> ... 3 more fields> ... 12 more fields]

In [5]:
dfv1.count

count at cmd4.sc:1

115 / 115

count at cmd4.sc:1

1 / 1

res4: Long = 1678005L

In [6]:
dfv2.count

count at cmd5.sc:1

24 / 24

count at cmd5.sc:1

1 / 1

res5: Long = 9447082L

In [24]:
assert(dfv1.select("id").distinct.count == dfv1.count)

count at cmd23.sc:1

115 / 115

count at cmd23.sc:1

1 / 1

count at cmd23.sc:1

115 / 115

count at cmd23.sc:1

1 / 1

In [7]:
dfv2.groupBy("id").count.groupBy("count").count.show

show at cmd6.sc:1

24 / 24

show at cmd6.sc:1

1 / 1

show at cmd6.sc:1

1 / 1

+-----+-------+
|count|  count|
+-----+-------+
|    2| 863400|
|    1|7681353|
|    5|    387|
|    3|   6677|
|    4|   3672|
|    8|     38|
|    6|    236|
|    7|     42|
|   12|      2|
|   11|      2|
|   10|      3|
|    9|     13|
|   15|      1|
|   13|      1|
|   22|      1|
|   18|      1|
+-----+-------+



### Validate Record Presence

Before equating values, simply check that the record ids are present in both datasets:

In [7]:
val df1 = dfv1.select($"id", $"sourceID", $"target.id".as("target_id"), $"disease.id".as("disease_id"))
    .transform(prefix("v1:")).dropDuplicates()
val df2 = dfv2.select($"id", $"sourceID", $"target.id".as("target_id"), $"disease.id".as("disease_id"))
    .transform(prefix("v2:")).dropDuplicates()
val dfj = df1.join(df2, df1("v1:id") === df2("v2:id"), "outer")
    .withColumn("status", 
        when($"v1:id".isNull && $"v2:id".isNotNull, "v2_only")
        .when($"v1:id".isNotNull && $"v2:id".isNull, "v1_only")
        .when($"v1:id".isNotNull && $"v2:id".isNotNull, "both")
        .otherwise("neither")
    )

defined function prefix
df1: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [v1:id: string, v1:sourceID: string ... 2 more fields]
df2: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [v2:id: string, v2:sourceID: string ... 2 more fields]
dfj: DataFrame = [v1:id: string, v1:sourceID: string ... 7 more fields]

In [8]:
dfj.groupBy($"v2:sourceID", $"status").count.show

show at cmd7.sc:1

24 / 24

show at cmd7.sc:1

115 / 115

show at cmd7.sc:1

...

show at cmd7.sc:1

1 / 1

show at cmd7.sc:1

1 / 1

+------------------+-------+-------+
|       v2:sourceID| status|  count|
+------------------+-------+-------+
|           uniprot|   both|  31555|
|         europepmc|v2_only|6913511|
|cancer_gene_census|   both|  96274|
|        slapenrich|   both|  74570|
|  expression_atlas|   both| 202648|
|            chembl|   both| 384754|
|         phenodigm|   both| 501788|
|      gwas_catalog|   both| 186237|
|               eva|   both| 103019|
|       eva_somatic|   both|   7879|
|          reactome|   both|  10989|
|    phewas_catalog|   both|  61947|
|  genomics_england|   both|  10550|
|    gene2phenotype|   both|   1586|
|uniprot_literature|   both|   4818|
|           intogen|   both|   2375|
|            crispr|   both|   1844|
|      gwas_catalog|v2_only|    484|
|   uniprot_somatic|   both|    284|
|            sysbio|   both|    408|
+------------------+-------+-------+
only showing top 20 rows



Of the two discrepancies above, the EPMC differences are expected since more data was processed in the v2 pipeline.  The GWAS catalog data however is a result of the UniProt -> Non-reference gene -> reference Ensembl id translation being broken in the v1 pipeline.  Prove the latter is true:

In [9]:
val dfx = dfj.filter($"v2:sourceID" === "gwas_catalog" && $"status" === "v2_only").persist()
def idx = dfx.select("v2:id").collect.map(_(0).asInstanceOf[String])
dfv2.filter($"id".isin(idx:_*)).groupBy("sourceID", "type").count.show

collect at cmd8.sc:2

24 / 24

collect at cmd8.sc:2

115 / 115

collect at cmd8.sc:2

...

24 / 24

show at cmd8.sc:3

1 / 1

+------------+-------------------+-----+
|    sourceID|               type|count|
+------------+-------------------+-----+
|gwas_catalog|genetic_association|  526|
+------------+-------------------+-----+



dfx: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [v1:id: string, v1:sourceID: string ... 7 more fields]
defined function idx

In [10]:
// Load the non-reference gene mapping
val dfnr = ss.read
      .option("header", "true")
      .csv(MAIN_RESOURCE_DIR.resolve("data").resolve("genes_with_non_reference_ensembl_ids_lkp.tsv").toString)
dfnr.show(3, false)

csv at cmd9.sc:3

1 / 1

show at cmd9.sc:4

1 / 1

+-----------+---------------+---------------+
|gene_symbol|reference      |alternate      |
+-----------+---------------+---------------+
|ABCB11     |ENSG00000073734|ENSG00000276582|
|AGER       |ENSG00000204305|ENSG00000237405|
|AGK        |ENSG00000006530|ENSG00000262327|
+-----------+---------------+---------------+
only showing top 3 rows



dfnr: DataFrame = [gene_symbol: string, reference: string ... 1 more field]

In [11]:
// Show that of the v2 gwas_catalog records not in the v1 results, they all resolve to 
// reference genes in the non-reference mapping:
dfv2
    .filter($"id".isin(idx:_*))
    .withColumn("has_nonref_id", $"target.id".isin(
        dfnr.select("reference").collect.map(_(0).asInstanceOf[String]):_*
    ))
    .groupBy("has_nonref_id").count.show

collect at cmd8.sc:2

1 / 1

collect at cmd10.sc:4

1 / 1

show at cmd10.sc:6

24 / 24

show at cmd10.sc:6

1 / 1

+-------------+-----+
|has_nonref_id|count|
+-------------+-----+
|         true|  526|
+-------------+-----+



### Duplicate Record Validation

In [ ]:
val df1 = dfv1.select("id")
val df2 = dfv2.select("id", "sourceID", "type")
def dupeIds = dfv1.select("id").join(
        dfv2.select("id", "sourceID", "type"), 
        Seq("id"), 
        "inner"
    ).groupBy("sourceID", "type", "id").count.filter($"count" > 1)
    .select("id").collect.map(_(0).asInstanceOf[String])
val dfidct = dfv2
    .filter($"id".isin(dupeIds:_*))
    .withColumn("target_disease", concat($"target.id", lit(":"), $"disease.id"))
    .groupBy("sourceID", "type", "id")
    .agg(countDistinct("target_disease").as("ct_tgt_dis"), count("id").as("ct"))
    .cache()

In [36]:
dfidct.filter($"ct_tgt_dis" === $"ct").groupBy("sourceID", "type")
    .agg(count("id").as("ctid"), sum("ct").as("ctrec")).show

show at cmd35.sc:2

1 / 1

show at cmd35.sc:2

1 / 1

+----------------+-------------------+----+-----+
|        sourceID|               type|ctid|ctrec|
+----------------+-------------------+----+-----+
|expression_atlas|     rna_expression| 136|  272|
|  phewas_catalog|genetic_association|4735|10405|
+----------------+-------------------+----+-----+



In [42]:
import org.apache.spark.sql.expressions.Window

import org.apache.spark.sql.expressions.Window

In [57]:
dfidct
    .filter($"ct_tgt_dis" === $"ct")
    .withColumn("rid", row_number.over(Window.partitionBy($"sourceID", $"type").orderBy($"ct_tgt_dis".desc)))
    .filter($"rid" <= 10)
    .show(20, false)

show at cmd56.sc:5

1 / 1

show at cmd56.sc:5

1 / 1

+----------------+-------------------+--------------------------------+----------+---+---+
|sourceID        |type               |id                              |ct_tgt_dis|ct |rid|
+----------------+-------------------+--------------------------------+----------+---+---+
|expression_atlas|rna_expression     |975717f23fabf725ae9428d3fd6288ca|2         |2  |1  |
|expression_atlas|rna_expression     |190e566aa22cd366bba1df7c5ace79a1|2         |2  |2  |
|expression_atlas|rna_expression     |a14c530e035d7ceb10f258551594ed5b|2         |2  |3  |
|expression_atlas|rna_expression     |72ed48343a39255b956add2ad5c5c7eb|2         |2  |4  |
|expression_atlas|rna_expression     |e1b5b5affdcdf36b8e3666e689b74e6a|2         |2  |5  |
|expression_atlas|rna_expression     |dab3d2e2f7430b49fc31cad8b8ff3e8d|2         |2  |6  |
|expression_atlas|rna_expression     |541ece3c99a3e74b90ed4408ffeb28de|2         |2  |7  |
|expression_atlas|rna_expression     |210399f7850d19fa6607d9fa888db051|2         |2  |8  |

#### Expression Atlas Issues

In [60]:
// Verify that All of the expression_atlas records correspond to one single issue
// with a study of Orphanet_586 and EFO_1000024
def geteaids = dfidct
    .filter($"ct_tgt_dis" === $"ct")
    .filter($"sourceID" === "expression_atlas")
    .select("id").distinct.collect.map(_.getString(0))
// This should be empty if there are any other EA issues
dfv2
    .filter($"id".isin(geteaids:_*))
    .filter(!$"disease.id".contains("Orphanet_586") && !$"disease.id".contains("EFO_1000024"))
    .select(
        $"id",
        $"target.id".as("target.id"),
        $"disease.id".as("disease.id"),
        $"sourceID".as("sourceID"),
        $"type".as("type"),
        to_json($"unique_association_fields")
    )
    .show(10, false)

collect at cmd59.sc:4

1 / 1

collect at cmd59.sc:4

1 / 1

show at cmd59.sc:16

...

4 / 4

show at cmd59.sc:16

19 / 19

+---+---------+----------+--------+----+----------------------------------------+
|id |target.id|disease.id|sourceID|type|structstojson(unique_association_fields)|
+---+---------+----------+--------+----+----------------------------------------+
+---+---------+----------+--------+----+----------------------------------------+



defined function geteaids

In [61]:
dfv2
    .filter($"sourceID" === "expression_atlas")
    .filter($"type" === "rna_expression")
    .filter($"target.id".contains("ENSG00000091137"))
    .filter($"disease.id".contains("Orphanet_586") || $"disease.id".contains("EFO_1000024"))
    .select(
        $"id",
        $"target.id",
        $"disease.id",
        to_json($"unique_association_fields")
    )
    .show(10, false)

show at cmd60.sc:12

1 / 1

show at cmd60.sc:12

4 / 4

show at cmd60.sc:12

19 / 19

+--------------------------------+---------------+------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|id                              |id             |id          |structstojson(unique_association_fields)                                                                                                                                                                                                                   |
+--------------------------------+---------------+------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|265f3ff9391061134e8d116c23f16aca|ENSG00000091137|EF

#### PheWAS Catalog Issues

In [62]:
dfv2
    .filter($"id".isin("46ecc60c84b1e31974ac6908cc4f8f65"))
    .select(
        $"id",
        $"target.id".as("target.id"),
        $"disease.id".as("disease.id"),
        $"sourceID".as("sourceID"),
        $"type".as("type"),
        to_json($"unique_association_fields")
    )
    .show(10, false)

show at cmd61.sc:11

1 / 1

show at cmd61.sc:11

4 / 4

show at cmd61.sc:11

19 / 19

+--------------------------------+---------------+-----------+--------------+-------------------+------------------------------------------------------------------------------------------------------------------------+
|id                              |target.id      |disease.id |sourceID      |type               |structstojson(unique_association_fields)                                                                                |
+--------------------------------+---------------+-----------+--------------+-------------------+------------------------------------------------------------------------------------------------------------------------+
|46ecc60c84b1e31974ac6908cc4f8f65|ENSG00000139155|EFO_0004269|phewas_catalog|genetic_association|{"cases":"1338","odds_ratio":"1.149","phenotype":"Abnormal electrocardiogram [ECG] [EKG]","datasource":"phewas_catalog"}|
|46ecc60c84b1e31974ac6908cc4f8f65|ENSG00000130204|EFO_0004269|phewas_catalog|genetic_association|{"cases":"1338","odds_ratio

In [64]:
dfv1
    .filter($"id" === "46ecc60c84b1e31974ac6908cc4f8f65")
    .select(
        $"id",
        $"target.id".as("target.id"),
        $"disease.id".as("disease.id"),
        $"sourceID".as("sourceID"),
        $"type".as("type"),
        to_json($"unique_association_fields")
    )
    .show(10, false)

show at cmd63.sc:11

1 / 1

show at cmd63.sc:11

4 / 4

show at cmd63.sc:11

20 / 20

show at cmd63.sc:11

90 / 90

+--------------------------------+---------------+-----------+--------------+-------------------+------------------------------------------------------------------------------------------------------------------------+
|id                              |target.id      |disease.id |sourceID      |type               |structstojson(unique_association_fields)                                                                                |
+--------------------------------+---------------+-----------+--------------+-------------------+------------------------------------------------------------------------------------------------------------------------+
|46ecc60c84b1e31974ac6908cc4f8f65|ENSG00000122641|EFO_0004269|phewas_catalog|genetic_association|{"cases":"1338","odds_ratio":"1.149","phenotype":"Abnormal electrocardiogram [ECG] [EKG]","datasource":"phewas_catalog"}|
+--------------------------------+---------------+-----------+--------------+-------------------+---------------------------

### Field Validation

Fields to validate:
- evidence.gene2variant.resource_score.{type, value}
- evidence.resource_score.{type, value}
- evidence.evidence_codes (array of string)
- target.id
- disease.id

In [18]:
val cols = Seq(
    col("id"),
    col("sourceID"),
    col("evidence.gene2variant.resource_score.type").as("g2vrs_type"),
    col("evidence.gene2variant.resource_score.value").as("g2vrs_value"),
    col("evidence.resource_score.type").as("ers_type"),
    col("evidence.resource_score.value").as("ers_value"),
    sort_array(col("evidence.evidence_codes")).as("evidence_codes"),
    col("target.id").as("target_id"),
    col("disease.id").as("disease_id")
)
val df1 = dfv1.select(cols:_*).transform(prefix("v1:")).dropDuplicates("v1:id")
val df2 = dfv2.select(cols:_*).transform(prefix("v2:")).dropDuplicates("v2:id")
val dfm = df1.join(df2, df1("v1:id") === df2("v2:id"), "inner").persist()

cols: Seq[org.apache.spark.sql.Column] = List(
  id,
  sourceID,
  evidence.gene2variant.resource_score.type AS `g2vrs_type`,
  evidence.gene2variant.resource_score.value AS `g2vrs_value`,
  evidence.resource_score.type AS `ers_type`,
  evidence.resource_score.value AS `ers_value`,
  sort_array(evidence.evidence_codes, true) AS `evidence_codes`,
  target.id AS `target_id`,
  disease.id AS `disease_id`
)
df1: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [v1:id: string, v1:sourceID: string ... 7 more fields]
df2: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [v2:id: string, v2:sourceID: string ... 7 more fields]
dfm: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [v1:id: string, v1:sourceID: string ... 16 more fields]

In [19]:
val dfsct = dfm.columns.map(_.split(":").last).toSet.toSeq.sortWith(_ < _).map { c => 
    dfm
        .withColumn("source_id", col("v1:sourceID"))
        .withColumn("field", lit(c))
        .withColumn("status", 
            when(col("v1:" + c).isNull && col("v2:" + c).isNull, "both_null")
            .when(col("v1:" + c).isNotNull && col("v2:" + c).isNull, "v2_null")
            .when(col("v1:" + c).isNull && col("v2:" + c).isNotNull, "v1_null")
            .otherwise("not_null")
        )
        .withColumn("equal", 
            col("status") === "both_null" || 
            (col("status") === "not_null" && col("v1:" + c) === col("v2:" + c))
        )
        .groupBy("source_id", "field", "status", "equal").count
}.reduce(_.union(_))

dfsct: DataFrame = [source_id: string, field: string ... 3 more fields]

In [10]:
dfsct.groupBy("field", "status", "equal").sum("count").show

show at cmd9.sc:1

115 / 115

show at cmd9.sc:1

24 / 24

show at cmd9.sc:1

...

show at cmd9.sc:1

9 / 9

show at cmd9.sc:1

1 / 1

+--------------+---------+-----+----------+
|         field|   status|equal|sum(count)|
+--------------+---------+-----+----------+
|    disease_id| not_null| true|   1677916|
|    disease_id| not_null|false|        89|
|      ers_type|both_null| true|   1263608|
|      ers_type| not_null| true|    414397|
|     ers_value|both_null| true|   1263608|
|     ers_value| not_null| true|    414395|
|     ers_value| not_null|false|         2|
|evidence_codes| not_null|false|   1263631|
|evidence_codes| not_null| true|    414374|
|    g2vrs_type| not_null| true|    377066|
|    g2vrs_type|both_null| true|   1300939|
|   g2vrs_value| not_null| true|    377066|
|   g2vrs_value|both_null| true|   1300939|
|            id| not_null| true|   1678005|
|      sourceID| not_null| true|   1678005|
|     target_id| not_null| true|   1673374|
|     target_id| not_null|false|      4631|
+--------------+---------+-----+----------+



In [20]:
dfsct.filter(!$"equal").orderBy("field", "source_id", "equal", "status").show(40)

show at cmd19.sc:1

115 / 115

show at cmd19.sc:1

24 / 24

show at cmd19.sc:1

...

1 / 1

19/12/17 20:21:00 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
19/12/17 20:21:00 WARN Executor: Managed memory leak detected; size = 33554432 bytes, TID = 603


show at cmd19.sc:1

9 / 9

+----------------+--------------+--------+-----+------+
|       source_id|         field|  status|equal| count|
+----------------+--------------+--------+-----+------+
|expression_atlas|    disease_id|not_null|false|    89|
|         intogen|     ers_value|not_null|false|     1|
|          sysbio|     ers_value|not_null|false|     1|
|          chembl|evidence_codes|not_null|false|384754|
|             eva|evidence_codes|not_null|false|103019|
|    gwas_catalog|evidence_codes|not_null|false|186237|
|       phenodigm|evidence_codes|not_null|false|501788|
|  phewas_catalog|evidence_codes|not_null|false| 56255|
|         uniprot|evidence_codes|not_null|false| 31555|
|  phewas_catalog|     target_id|not_null|false|  4631|
+----------------+--------------+--------+-----+------+



In [23]:
dfv1.groupBy("id").count.groupBy("count").count.show

show at cmd22.sc:1

115 / 115

show at cmd22.sc:1

1 / 1

show at cmd22.sc:1

1 / 1

+-----+-------+
|count|  count|
+-----+-------+
|    1|1678005|
+-----+-------+



In [26]:
dfmid
    .groupBy("sourceID", "type", "id").count.filter($"count" > 1)
    .groupBy("sourceID", "type").count.show

show at cmd25.sc:4

24 / 24

show at cmd25.sc:4

115 / 115

show at cmd25.sc:4

1 / 1

show at cmd25.sc:4

1 / 1

+----------------+-------------------+-----+
|        sourceID|               type|count|
+----------------+-------------------+-----+
|    gwas_catalog|genetic_association| 5126|
|  phewas_catalog|genetic_association| 4812|
|          chembl|         known_drug|11890|
|             eva|genetic_association|  214|
|expression_atlas|     rna_expression|  136|
|         intogen|   somatic_mutation|  113|
|  gene2phenotype|genetic_association|   11|
|          sysbio|   affected_pathway|    1|
|         progeny|   affected_pathway|   11|
+----------------+-------------------+-----+



In [12]:
dfm
    .filter($"v1:sourceID" === "phewas_catalog")
    .filter($"v1:target_id" =!= "v2:target_id")
    .select("v1:id", "v1:target_id", "v2:target_id")
    .show(10, false)

show at cmd11.sc:5

1 / 1

+--------------------------------+---------------+---------------+
|v1:id                           |v1:target_id   |v2:target_id   |
+--------------------------------+---------------+---------------+
|0001c78e77ccaa1f2b43c7ab238ee209|ENSG00000116675|ENSG00000135966|
|0002a5aa8d98dcb54b65d45774731e5a|ENSG00000163406|ENSG00000163406|
|000506e394e0c319dc8edd4003f57334|ENSG00000163110|ENSG00000163110|
|0007448ea41ce39de9b0301cbaae746d|ENSG00000140285|ENSG00000140285|
|0008f59c44bcc858cd6e50934dc03963|ENSG00000135541|ENSG00000153930|
|000c1b2bafea76a6c119f6c39f8e77bf|ENSG00000106633|ENSG00000106633|
|000fd562c431e8f8468a3069cd9c077b|ENSG00000146833|ENSG00000146833|
|00104b3acf627e4c57887d04d2c1d6bb|ENSG00000164270|ENSG00000164270|
|0013296937f3ee707e22b4c12f5106f4|ENSG00000174437|ENSG00000174437|
|00134ef10bae73da7111d3d65c484389|ENSG00000078070|ENSG00000078070|
+--------------------------------+---------------+---------------+
only showing top 10 rows



In [5]:
ss.read.json(EXTRACT_DIR.resolve("evidence_raw.json").toString)
    .filter($"sourceID" === "phewas_catalog")
    .filter($"target.id".contains("ENSG00000135966"))
    .filter($"disease.id".contains("HP_0001919"))
    .select(
        "target.id",
        "disease.id",
        "evidence.evidence_codes",
        "evidence.variant2disease.evidence_codes",
        "evidence.gene2variant.evidence_codes",
        "evidence.target2drug.evidence_codes",
        "evidence.drug2clinic.evidence_codes",
        "evidence.biological_model.evidence_codes",
        "evidence.gene2variant.functional_consequence"
    )
    .show(10, false)

json at cmd4.sc:1

178 / 178

show at cmd4.sc:16

1 / 1

show at cmd4.sc:16

...

show at cmd4.sc:16

100 / 100

show at cmd4.sc:16

53 / 53

+----------------------------------------------+-----------------------------------------+--------------+-----------------------------------+--------------------------------------------+--------------+--------------+--------------+-----------------------------------------+
|id                                            |id                                       |evidence_codes|evidence_codes                     |evidence_codes                              |evidence_codes|evidence_codes|evidence_codes|functional_consequence                   |
+----------------------------------------------+-----------------------------------------+--------------+-----------------------------------+--------------------------------------------+--------------+--------------+--------------+-----------------------------------------+
|http://identifiers.org/ensembl/ENSG00000135966|http://purl.obolibrary.org/obo/HP_0001919|null          |[http://identifiers.org/eco/PheWAS]|[http://purl.obolibrary.org/obo/ECO_0